In [1]:
import numpy as np
from scipy import linalg as lg
import time

def levenberg_marquardt(tensor, rank, seed=42, tol=1e-8, max_iter=100, lambda_init=1.0, lambda_scale=2.0, verbose=True):
    dimensions = tensor.shape
    ndim = len(dimensions)

    jacob_size = int(np.sum(dimensions) * rank)

    rng = np.random.RandomState(seed)
    factors = [rng.rand(dimensions[i], rank) for i in range(ndim)]

    lambda_param = lambda_init

    pos = [0]
    for i in range(ndim - 1):
        pos.append(pos[i] + dimensions[i] * rank)

    error = float('inf')
    iterations = 0
    
    while error > tol and iterations < max_iter:
        layers = compute_layers(ndim, factors)
        W = compute_W(ndim, layers)
        
        JJ = compute_JJ(ndim, W, pos, jacob_size, dimensions, rank, factors)
        JA = compute_JA(ndim, factors, rank, dimensions, W, jacob_size)
        JT = compute_JT(ndim, factors, rank, dimensions, jacob_size, tensor)
        
        b = JA - JT
        
        D_s = np.sqrt(np.diag(JJ))
        D_s_i = 1 / D_s
        
        b_scaled = D_s_i * b
        JJ_scaled = D_s_i.reshape((-1, 1)) * JJ * D_s_i
        
        T, U = lg.hessenberg(JJ_scaled, calc_q=True)
        b_transformed = U.T @ b_scaled
        
        best_lambda = None
        best_error = float('inf')
        
        for lambda_multiplier in [1.0, lambda_scale]:
            cur_lambda = lambda_param * lambda_multiplier
            
            new_T = T + cur_lambda * np.eye(jacob_size)
            y = lg.solve(new_T, b_transformed, assume_a='her')
            delta = U @ y
            delta_scaled = D_s_i * delta
            
            new_factors = update_factors(ndim, factors, delta_scaled, rank, dimensions)
            cur_error = compute_approximation_error(tensor, new_factors, rank)
            
            if cur_error < best_error:
                best_lambda = cur_lambda
                best_error = cur_error
                best_factors = new_factors
        
        if best_error < error:
            factors = best_factors
            error = best_error
            lambda_param = best_lambda / lambda_scale
        else:
            lambda_param *= lambda_scale
        
        iterations += 1
        
        if verbose and iterations % 5 == 0:
            print(f'Iteration {iterations}/{max_iter}, Error: {error:.8f}, Lambda: {lambda_param:.6f}')
    
    if verbose:
        print(f'Final error: {error} after {iterations} iterations')
    
    return factors, error, iterations

def compute_layers(ndim, factors):
    layers = []
    first_layer = []
    for n in range(ndim):
        first_layer.append(factors[n].T @ factors[n])
    layers.append(first_layer)
    
    for n in range(1, ndim - 1):
        layer = []
        for r in range(ndim - n):
            layer.append(layers[n - 1][r] * layers[0][r + n])
        layers.append(layer)
    
    return layers

def compute_W(ndim, layers):
    W = []
    
    for row in range(ndim):
        W_row = []
        
        start_layer = row - 1
        end_layer = ndim - row - 2
        
        if start_layer >= 0:
            W_col = layers[start_layer][0]
            if end_layer >= 0:
                W_col = W_col * layers[end_layer][-1]
        else:
            if end_layer >= 0:
                W_col = layers[end_layer][-1]
            else:
                W_col = np.eye(len(layers[0][0]))
        W_row.append(W_col)
        
        for col in range(row + 1, ndim):
            start_layer = row - 1
            middle_layer = col - row - 2
            end_layer = ndim - col - 2
            
            W_col = None
            
            if start_layer >= 0:
                W_col = layers[start_layer][0]
                
            if middle_layer >= 0:
                if W_col is not None:
                    W_col = W_col * layers[middle_layer][row + 1]
                else:
                    W_col = layers[middle_layer][row + 1]
                    
            if end_layer >= 0:
                if W_col is not None:
                    W_col = W_col * layers[end_layer][-1]
                else:
                    W_col = layers[end_layer][-1]
            
            if W_col is None:
                W_col = np.eye(len(layers[0][0]))
            
            W_row.append(W_col)
        
        W.append(W_row)
    
    return W

def compute_JJ(ndim, W, pos, jacob_size, dimensions, rank, factors):
    JJ = np.zeros((jacob_size, jacob_size))
    
    for row in range(ndim):
        BB = W[row][0]
        for r in range(dimensions[row]):
            now_pos = pos[row] + r * rank
            JJ[now_pos:now_pos+rank, now_pos:now_pos+rank] = BB
    
    for alpha in range(ndim):
        now_alpha = pos[alpha]
        for i in range(alpha + 1, ndim):
            now_i = pos[i]
            
            now_W = W[alpha][i - alpha]
            for betta in range(dimensions[alpha]):
                now_a_b = now_alpha + betta * rank
                for j in range(dimensions[i]):
                    now_i_j = now_i + j * rank
                    for gamma in range(rank):
                        now_a_b_g = now_a_b + gamma
                        for k in range(rank):
                            now_i_j_k = now_i_j + k
                            val = now_W[gamma][k] * factors[i][j][gamma] * factors[alpha][betta][k]
                            JJ[now_a_b_g, now_i_j_k] = val
                            JJ[now_i_j_k, now_a_b_g] = now_W[k][gamma] * factors[i][j][gamma] * factors[alpha][betta][k]
    
    return JJ

def compute_JA(ndim, factors, rank, dimensions, W, jacob_size):
    JA = np.zeros(jacob_size)
    
    for i in range(ndim):
        BB = W[i][0]
        pos_i = int(np.sum(dimensions[:i]) * rank)
        
        for j in range(dimensions[i]):
            pos_j = pos_i + j * rank
            for k in range(rank):
                pos_k = pos_j + k
                val = 0
                for r in range(rank):
                    val += BB[k, r] * factors[i][j, r]
                JA[pos_k] = val
    
    return JA

def compute_JT(ndim, factors, rank, dimensions, jacob_size, tensor):
    JT = np.zeros(jacob_size)
    
    for mode in range(ndim):
        start_pos = int(np.sum(dimensions[:mode]) * rank)
        
        tensor_mat = unfold(tensor, mode)
        
        kr_product = None
        for n in range(ndim):
            if n != mode:
                if kr_product is None:
                    kr_product = factors[n]
                else:
                    kr_product = lg.khatri_rao(kr_product, factors[n])
        
        mode_product = tensor_mat @ kr_product
        
        for i in range(dimensions[mode]):
            JT[start_pos + i*rank:start_pos + (i+1)*rank] = mode_product[i, :]
    
    return JT

def unfold(tensor, mode):
    dims = tensor.shape
    n_dim = len(dims)
    
    new_order = [mode] + [i for i in range(n_dim) if i != mode]
    
    transposed = np.transpose(tensor, new_order)
    
    mode_dim = dims[mode]
    other_dims = np.prod([dims[i] for i in range(n_dim) if i != mode])
    
    return transposed.reshape(mode_dim, other_dims)

def update_factors(ndim, factors, delta, rank, dimensions):
    new_factors = []
    for i in range(ndim):
        position = int(np.sum(dimensions[:i]) * rank)
        factor = factors[i].copy()
        for j in range(dimensions[i]):
            delta_slice = delta[position + j * rank:position + (j+1)*rank]
            factor[j, :] = factors[i][j, :] - delta_slice
        new_factors.append(factor)
    return new_factors

def compute_approximation_error(tensor, factors, rank):
    approximation = reconstruct_cp_tensor(factors, tensor.shape)
    error = np.linalg.norm(tensor.reshape(-1) - approximation.reshape(-1)) / np.linalg.norm(tensor.reshape(-1))
    return error

def reconstruct_cp_tensor(factors, shape):
    ndim = len(shape)
    rank = factors[0].shape[1]
    result = np.zeros(shape)
    
    for r in range(rank):
        component_vectors = [factors[n][:, r] for n in range(ndim)]
        outer_product = component_vectors[0]
        for n in range(1, ndim):
            outer_product = np.multiply.outer(outer_product, component_vectors[n])
        result += outer_product
    
    return result

In [2]:
sizes = np.array((20, 15, 25, 20))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = np.sin(np.sum(I))

st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=4, 
    seed=42, 
    tol=1e-6, 
    max_iter=50,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/50, Error: 0.99970715, Lambda: 0.500000
Iteration 10/50, Error: 0.89114722, Lambda: 4.000000
Iteration 15/50, Error: 0.50418349, Lambda: 0.500000
Iteration 20/50, Error: 0.00016381, Lambda: 0.015625
Final error: 4.000355995868131e-07 after 22 iterations

Levenberg Marquardt time: 2.17621
Final approximation error: 4.000355995868131e-07
Number of iterations: 22


In [3]:
sizes = np.array((20, 15, 25, 20))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = np.sum(I)

st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=5, 
    seed=42, 
    tol=1e-8, 
    max_iter=50,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/50, Error: 82.44975705, Lambda: 0.062500
Iteration 10/50, Error: 0.23095409, Lambda: 0.001953
Iteration 15/50, Error: 0.05908414, Lambda: 0.015625
Iteration 20/50, Error: 0.05908414, Lambda: 0.500000
Iteration 25/50, Error: 0.01630010, Lambda: 0.062500
Iteration 30/50, Error: 0.00871957, Lambda: 0.500000
Iteration 35/50, Error: 0.00277364, Lambda: 0.125000
Iteration 40/50, Error: 0.00043048, Lambda: 0.003906
Iteration 45/50, Error: 0.00000119, Lambda: 0.000122
Final error: 3.08621111713618e-09 after 47 iterations

Levenberg Marquardt time: 4.95784
Final approximation error: 3.08621111713618e-09
Number of iterations: 47


In [4]:
sizes = np.array((10, 10, 10, 10, 10))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = 1 / (1 + np.sum(I))

st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=5, 
    seed=42, 
    tol=1e-8, 
    max_iter=50,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/50, Error: 0.05448090, Lambda: 0.031250
Iteration 10/50, Error: 0.01991906, Lambda: 0.015625
Iteration 15/50, Error: 0.00834507, Lambda: 0.007812
Iteration 20/50, Error: 0.00494133, Lambda: 0.001953
Iteration 25/50, Error: 0.00269260, Lambda: 0.000977
Iteration 30/50, Error: 0.00214876, Lambda: 0.000977
Iteration 35/50, Error: 0.00155149, Lambda: 0.000977
Iteration 40/50, Error: 0.00095624, Lambda: 0.000488
Iteration 45/50, Error: 0.00083300, Lambda: 0.000061
Iteration 50/50, Error: 0.00080817, Lambda: 0.000031
Final error: 0.000808168016390968 after 50 iterations

Levenberg Marquardt time: 2.46438
Final approximation error: 0.000808168016390968
Number of iterations: 50


In [5]:
sizes = np.array((10, 25, 20, 25, 20))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = np.sin(np.sum(I))

In [6]:
st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=5, 
    seed=42, 
    tol=1e-8, 
    max_iter=50,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/50, Error: 1.00000456, Lambda: 0.125000
Iteration 10/50, Error: 0.97752547, Lambda: 2.000000
Iteration 15/50, Error: 0.97315808, Lambda: 32.000000
Iteration 20/50, Error: 0.85655311, Lambda: 8.000000
Iteration 25/50, Error: 0.51625433, Lambda: 0.250000
Iteration 30/50, Error: 0.00037079, Lambda: 0.007812
Final error: 1.7328390868629151e-09 after 34 iterations

Levenberg Marquardt time: 9.93003
Final approximation error: 1.7328390868629151e-09
Number of iterations: 34


In [7]:
sizes = np.array((10,10,10,10,10,10))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = np.sin(np.sum(I))

st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=6, 
    seed=42, 
    tol=1e-8, 
    max_iter=500,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/500, Error: 0.98885358, Lambda: 0.250000
Iteration 10/500, Error: 0.98885358, Lambda: 8.000000
Iteration 15/500, Error: 0.95872532, Lambda: 8.000000
Iteration 20/500, Error: 0.93246064, Lambda: 8.000000
Iteration 25/500, Error: 0.91356307, Lambda: 8.000000
Iteration 30/500, Error: 0.66843300, Lambda: 0.250000
Iteration 35/500, Error: 0.46192868, Lambda: 0.062500
Iteration 40/500, Error: 0.27858380, Lambda: 0.007812
Iteration 45/500, Error: 0.21996309, Lambda: 0.000244
Iteration 50/500, Error: 0.19263726, Lambda: 0.000122
Iteration 55/500, Error: 0.16441447, Lambda: 0.000061
Iteration 60/500, Error: 0.08503340, Lambda: 0.000061
Iteration 65/500, Error: 0.00000887, Lambda: 0.000004
Final error: 1.9758274389426187e-09 after 68 iterations

Levenberg Marquardt time: 11.70618
Final approximation error: 1.9758274389426187e-09
Number of iterations: 68


In [8]:
sizes = np.array((4, 32, 48, 96))
tensor = np.zeros(sizes)
for I in np.ndindex(*sizes):
    tensor[I] = np.sin(np.sum(I))
    
st = time.time()
factors, error, iterations = levenberg_marquardt(
    tensor, 
    rank=4, 
    seed=42, 
    tol=1e-8, 
    max_iter=500,
    verbose=True
)
print()
print(f"Levenberg Marquardt time: {time.time() - st:.5f}")
print(f"Final approximation error: {error}")
print(f"Number of iterations: {iterations}")

Iteration 5/500, Error: 0.99988289, Lambda: 0.500000
Iteration 10/500, Error: 0.78147706, Lambda: 1.000000
Iteration 15/500, Error: 0.00325731, Lambda: 0.062500
Iteration 20/500, Error: 0.00000000, Lambda: 0.001953
Final error: 6.996865399392117e-10 after 20 iterations

Levenberg Marquardt time: 6.94014
Final approximation error: 6.996865399392117e-10
Number of iterations: 20
